If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [1]:
!pip install transformers datasets nlp

     |█▊                              | 10 kB 26.1 MB/s eta 0:00:01

     |████████████████████████████████| 192 kB 4.6 MB/s 


     |██████▎                         | 327 kB 8.5 MB/s eta 0:00:01

     |███████████████▉                | 819 kB 8.5 MB/s eta 0:00:01

     |███████████████████████████     | 1.4 MB 8.5 MB/s eta 0:00:01

...

     |████████████████████████████████| 69 kB 5.2 MB/s 


     |████████████████████████████████| 243 kB 17.6 MB/s 


  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.56.2


    Uninstalling tqdm-4.56.2:


      Successfully uninstalled tqdm-4.56.2


# Fine-tuning a model on a text classification task

In [2]:
task = "test"
model_checkpoint = "xlm-roberta-large"
batch_size = 3

## Loading the dataset

In [3]:
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

import string
import re
import os

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [4]:
data = pd.read_csv('../input/newsclass01/train.csv', index_col=0)
data = data.rename({'title': 'sentence1', 'text': 'sentence2', 'source': 'label'}, axis=1)
data.index = data.index.rename('idx')
data

,sentence1,sentence2,images,label
idx,,,,
1,Міністри Зеленського лишають мільйони пацієнті...,Понад півтора мільйони пацієнтів із тяжкими ді...,NaN,5
2,В АМКУ звертаються щодо незаконної державної д...,За 15 років державна підтримка виробництва ене...,fd92a3dd-1109-49d8-8f5f-eeed72da22ef.png,6
5,У Польщі виявили майже 16 тисяч нових хворих н...,У Польщі виявили 15 698 нових хворих на корона...,f4a284d0-9bb7-4910-8a62-a7bfd0ec29b6.png,2
9,Закриття міжбанку: гривня трохи ослабла,2 жовтня котирування гривні до долара на закри...,NaN,3
10,"У Москві помер український шахіст із бази ""Мир...",У п'ятницю низка російських ЗМІ повідомили про...,"2cbb2339-6dd2-4000-a53c-225ec7aad892.png,d5233...",0
...,...,...,...,...
32366,"""Укрзалізниця"" просить Раду у бюджеті-2021 зак...","""Укрзалізниця"" закликає парламент закласти у д...",NaN,3
32367,Сніг та 20-градусні морози протримаються до ви...,"У вівторок, 16 лютого, в Україні буде без істо...",NaN,2
32368,"Україна хоче оподаткувати Google, Facebook та ...","Гетманцев розповів, як Україна оподатковуватим...",NaN,6


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16177 entries, 1 to 32371
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentence1  16177 non-null  object
 1   sentence2  16177 non-null  object
 2   images     7223 non-null   object
 3   label      16177 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 631.9+ KB


In [6]:
test_data = pd.read_csv('../input/newsclass01/test_without_target.csv', index_col=0)
test_data = test_data.drop(columns=['images'], axis=1).rename({'title': 'sentence1', 'text': 'sentence2', 'source': 'label'}, axis=1)
test_data.index = test_data.index.rename('idx')
test_data

,sentence1,sentence2
idx,,
0,Падіння економіки в Україні не є критичним - ЄБРР,Падіння економічного рівня в Україні не є крит...
3,Геймерам віддають безкоштовно шутер Rage 2 та ...,Шутер Rage 2 віддають безкоштовно в EGS / фото...
4,Записки из Луганска: привычка быть виноватыми ...,"""Ты печешь?"" – спрашивает у меня подруга, пере..."
6,Індекс Dow Jones вперше перевищив 30 тисяч пун...,На тлі новин щодо передачі влади у США та вакц...
7,Трамп і Меркель обговорили підтримку економічн...,Президент США Дональд Трамп і канцлерка Німечч...
...,...,...
32362,США хочуть закупити ще 100 млн доз вакцини про...,Президент США Джо Байден доручив Міністерству ...
32363,Вчителька вдарила школярку по голові і відпові...,У Первомайському ліцеї №7 Харківської області ...
32364,"""Не розслабляємося"": українців просять святкув...",Президент Володимир Зеленський попросив україн...


In [7]:
X, y = data.drop(columns=['label', 'images']), data['label']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [9]:
import datasets
from datasets import Dataset, DatasetDict, load_metric

dataset = datasets.dataset_dict.DatasetDict({'train': Dataset.from_pandas(pd.concat([X, y], axis=1), split='train'), 'validation': Dataset.from_pandas(pd.concat([X_test, y_test], axis=1), split='validation'), 'test': Dataset.from_pandas(test_data, split='test')})
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 16177
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 5339
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'idx'],
        num_rows: 16183
    })
})

You can call its `compute` method with your predictions and labels directly and it will return a dictionary with the metric(s) value:

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [10]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [11]:
tokenizer("Понад півтора мільйони пацієнтів із тяжкими", "Сніг та 20-градусні морози протримаються.")

{'input_ids': [0, 901, 42554, 49546, 33891, 147308, 89, 179387, 4459, 11849, 48878, 827, 2, 2, 589, 894, 476, 489, 13484, 12315, 105, 47279, 177640, 89, 591, 58070, 25863, 5, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

To preprocess our dataset, we will thus need the names of the columns containing the sentence(s). The following dictionary keeps track of the correspondence task to column names:

In [12]:
task_to_keys = {
    "test": ("sentence1", "sentence2"),
}

We can double check it does work on our current dataset:

In [13]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Text: {dataset['train'][sentence1_key][0]}")
else:
    print(f"Text: {dataset['train'][sentence1_key][0]}")
    print(f"Title: {dataset['train'][sentence2_key][0]}")

Text: Міністри Зеленського лишають мільйони пацієнтів без ліків
Title: Понад півтора мільйони пацієнтів із тяжкими діагнозами ризикують залишитись без ліків у 2020 році, бо МОЗ блокує закупівлі.
Заблоковані рекордні майже 10 мільярдів гривень у бюджеті на закупівлю ліків та медвиробів.
Ключові документи не підписувалися 4 місяці.
З голими руками і неадекватними заявами міністра Ємця: як Україна зустрічає коронавірус
І це в час, коли у світі пандемія коронавірусу, дефіцит багатьох ліків, країни стають в чергу на постачання і рахунок іде не на дні, а на години.
Натомість Україна в цю чергу ще навіть не стала.
З листопада 2019 року був готовий документ – перелік ліків і медвиробів для закупівель – і він весь цей час валявся для підпису на столі то в одного, то в іншого міністра.
Зараз він досі не ухвалений урядом.
Більше того, в міністерстві досі не призначений заступник міністра, який опікуватиметься закупівлями, не працює спеціальна робоча група, без якої закупівлі стоять, – в МОЗ не зр

We can them write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model.

In [14]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [15]:
preprocess_function(dataset['train'][:5])

{'input_ids': [[0, 135867, 89, 111675, 11188, 53737, 14632, 147308, 89, 179387, 1093, 11080, 10654, 2, 2, 901, 42554, 49546, 33891, 147308, 89, 179387, 4459, 11849, 48878, 827, 96417, 476, 122387, 1281, 50641, 14815, 177019, 4401, 1093, 11080, 10654, 84, 11075, 18122, 4, 3577, 1435, 77333, 21059, 10403, 240978, 5, 829, 76591, 44008, 70428, 894, 52503, 209, 166065, 790, 57321, 84, 12358, 260, 29, 40378, 790, 5235, 11080, 10654, 489, 17792, 2827, 8492, 790, 5, 208856, 260, 30560, 77, 99338, 122183, 201, 156440, 5, 1522, 17510, 5593, 94520, 189, 234238, 10606, 22263, 1281, 6, 61037, 27733, 130, 6218, 12, 1443, 28105, 71866, 13205, 211247, 59, 4335, 20017, 2144, 2157, 49, 2707, 4, 8790, 84, 60755, 36571, 30462, 3660, 211247, 6207, 97428, 2871, 4, 1757, 21305, 86463, 90165, 11080, 10654, 4, 51550, 6716, 14632, 49, 110824, 29, 224965, 189, 77579, 66280, 77, 29, 58089, 4, 252, 29, 3563, 5, 672, 328, 130122, 28105, 49, 64517, 110824, 1091, 24027, 77, 18637, 5, 1522, 56749, 3640, 4509, 14571, 6

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [16]:
encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2'],
        num_rows: 16177
    })
    validation: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2'],
        num_rows: 5339
    })
    test: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'sentence1', 'sentence2'],
        num_rows: 16183
    })
})

In [17]:
encoded_dataset["validation"][0]

{'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about sentence classification, we use the `AutoModelForSequenceClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which is always 2, except for STS-B which is a regression problem and MNLI where we have 3 labels):

In [18]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = len(data['label'].unique())
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
model

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
     

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define two more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [19]:
metric_name = "f1"

args = TrainingArguments(
    "test-RoBERTa",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay. Since the best model might not be the one at the end of training, we ask the `Trainer` to load the best model it saved (according to `metric_name`) at the end of training.

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits (our just squeeze the last axis in the case of STS-B):

In [20]:
from sklearn.metrics import f1_score

def compute_metrics(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=1)

    f1 = f1_score(labels, predictions, average='macro')
    return {
        'f1': f1,
    }

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [21]:
validation_key = "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

W&B installed but not logged in. Run `wandb login` or set the WANDB_API_KEY env variable.


You might wonder why we pass along the `tokenizer` when we already preprocessed our data. This is because we will use it once last time to make all the samples we gather the same length by applying padding, which requires knowing the model's preferences regarding padding (to the left or right? with which token?). The `tokenizer` has a pad method that will do all of this right for us, and the `Trainer` will use it. You can customize this part by defining and passing your own `data_collator` which will receive the samples like the dictionaries seen above and will need to return a dictionary of tensors.

We can now finetune our model by just calling the `train` method:

In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss


We can check with the `evaluate` method that our `Trainer` did reload the best model properly (if it was not the last one):

In [ ]:
trainer.evaluate()

In [ ]:
res = trainer.predict(encoded_dataset['test'])
res

In [ ]:
df_res = pd.DataFrame(np.argmax(res.predictions, axis=1), index=test_data.index)
df_res.index = df_res.index.rename('Id')
df_res.columns = ['Predicted']
df_res

In [ ]:
df_res.to_csv('./submission-xlm-roberta-large-v1.csv')

In [ ]:
args = TrainingArguments(
    "test-RoBERTa",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [ ]:
from transformers import pipeline

pipeline('feature-extraction', model=model, tokenizer=tokenizer)

In [ ]:
import shutil

shutil.copytree('./test-RoBERTa', '/content/drive/MyDrive/contest/models/RoBERTa-large')

<a href="./test-RoBERTa"> Download File </a>

## Hyperparameter search

The `Trainer` supports hyperparameter search using [optuna](https://optuna.org/) or [Ray Tune](https://docs.ray.io/en/latest/tune/). For this last section you will need either of those libraries installed, just uncomment the line you want on the next cell and run it.

In [ ]:
# ! pip install optuna
# ! pip install ray[tune]

During hyperparameter search, the `Trainer` will run several trainings, so it needs to have the model defined via a function (so it can be reinitialized at each new run) instead of just having it passed. We jsut use the same function as before:

In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

And we can instantiate our `Trainer` like before:

In [ ]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

The method we call this time is `hyperparameter_search`. Note that it can take a long time to run on the full dataset for some of the tasks. You can try to find some good hyperparameter on a portion of the training dataset by replacing the `train_dataset` line above by:
```python
train_dataset = encoded_dataset["train"].shard(index=1, num_shards=10) 
```
for 1/10th of the dataset. Then you can run a full training on the best hyperparameters picked by the search.

In [ ]:
best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")

The `hyperparameter_search` method returns a `BestRun` objects, which contains the value of the objective maximized (by default the sum of all metrics) and the hyperparameters it used for that run.

In [ ]:
best_run

You can customize the objective to maximize by passing along a `compute_objective` function to the `hyperparameter_search` method, and you can customize the search space by passing a `hp_space` argument to `hyperparameter_search`. See this [forum post](https://discuss.huggingface.co/t/using-hyperparameter-search-in-trainer/785/10) for some examples.

To reproduce the best training, just set the hyperparameters in your `TrainingArgument` before creating a `Trainer`:

In [ ]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()

Don't forget to [update your model](https://huggingface.co/transformers/model_sharing.html) on the [🤗 Model Hub](https://huggingface.co/models). You can then use it only to generate results like the one shown in the first picture of this notebook!